In [118]:
pip install openai==0.28

Note: you may need to restart the kernel to use updated packages.


In [16]:
import openai

with open("config/OpenAPI_Key.txt", "r") as f:
  openai.api_key = ' '.join(f.readlines())

In [17]:
import os, json, ast
from tenacity import retry, wait_random_exponential

In [18]:
os.environ["OPENAI_API_KEY"] = openai.api_key

In [120]:
import pandas as pd
df = pd.read_csv('data/final.csv', encoding='unicode_escape')
df.head(10)

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city),Description
0,https://www.zomato.com/bangalore/rosewood-inte...,"19/1, New Timberyard Layout, Beside Satellite ...",Rosewood International Hotel - Bar & Restaurant,No,No,3.6/5,8,+91 9731716688,Mysore Road,Casual Dining,NaN,"North Indian, South Indian, Andhra, Chinese",800,"[('Rated 5.0', 'RATED\næ Awesome food ??Great ...",[],Buffet,Banashankari,NaN
1,https://www.zomato.com/bangalore/penthouse-caf...,"1, 30th Main Road, 3rd Stage, Banashankari, Ba...",Penthouse Cafe,Yes,No,4.0/5,324,+91 8884135549,Banashankari,Cafe,"Pizza, Mocktails, Coffee, Nachos, Salad, Pasta...","Cafe, Italian, Continental",700,"[('Rated 3.0', ""RATED\næ I had been to this pl...",NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.zomato.com/bangalore/empire-restau...,"4 & 5, Green Orchards Layout, 1st Cross, Banne...",Empire Restaurant,Yes,No,4.3/5,3178,080 49653239,Bannerghatta Road,Casual Dining,"Chicken Satay, Mutton Brain, Egg Biryani, Pane...","North Indian, Mughlai, South Indian, Chinese",750,"[('Rated 5.0', 'RATED\næ Buffet Lunch @ Empire...","['Kerala Parota', 'Chicken Fried Rice', 'Tando...",Buffet,Bannerghatta Road,NaN
4,https://www.zomato.com/bangalore/starbucks-ban...,"Vega City, Dollars Colony, Bannerghatta Road, ...",Starbucks,No,No,3.9/5,52,NaN,Bannerghatta Road,Cafe,NaN,"Cafe, Desserts",700,"[('Rated 5.0', ""RATED\næ The place shuts at 11...",[],Cafes,Bannerghatta Road,NaN
5,https://www.zomato.com/bangalore/timepass-dinn...,"37, 5-1, 4th Floor, Bosco Court, Gandhi Bazaar...",Timepass Dinner,Yes,No,3.8/5,286,+91 9980040002,Basavanagudi,Casual Dining,"Onion Rings, Pasta, Kadhai Paneer, Salads, Sal...",North Indian,600,"[('Rated 3.0', 'RATED\næ Food 3/5\nAmbience 3/...",[],Buffet,Basavanagudi,NaN
6,https://www.zomato.com/bangalore/alchemy-coffe...,"51 Ground Floor, 5th Main Road, 36th Cross Roa...",Alchemy Coffee Roasters,No,No,4.2/5,704,+91 8762473937,Jayanagar,Cafe,"Coffee, Fries, Sandwich, Pizza, Pancakes, Past...",Cafe,500,"[('Rated 4.0', 'RATED\næ Alchemy coffee roaste...",NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,e taste of the food ... nothing special about ...,æ('Rated 4.0',æ'RATED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,æ Wanted to come here from quiet sometime n ya,æhere I was trying some no cheese bread basket,æmushroom soup and South Indian filter coffee....,æ('Rated 3.0',æ'RATED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [121]:
for index, row in df.head().iterrows():
    print("===========================")
    print("online_order:", row['online_order'], "\nbook_table:",row['book_table'],"\nvotes:", row['votes'], "\nlisted_in(type):",row['listed_in(type)'], "\nlisted_in(city):",row['listed_in(city)'], "\napprox_cost(for two people):",row['approx_cost(for two people)'], "\ncuisines:",row['cuisines'],"\ndish_liked:",row['dish_liked'])
    print("===========================")

online_order: No 
book_table: No 
votes: 8 
listed_in(type): Buffet 
listed_in(city): Banashankari 
approx_cost(for two people): 800 
cuisines: North Indian, South Indian, Andhra, Chinese 
dish_liked: nan
online_order: Yes 
book_table: No 
votes: 324 
listed_in(type): nan 
listed_in(city): nan 
approx_cost(for two people): 700 
cuisines: Cafe, Italian, Continental 
dish_liked: Pizza, Mocktails, Coffee, Nachos, Salad, Pasta, Sandwiches
online_order: nan 
book_table: nan 
votes: nan 
listed_in(type): nan 
listed_in(city): nan 
approx_cost(for two people): nan 
cuisines: nan 
dish_liked: nan
online_order: Yes 
book_table: No 
votes: 3178 
listed_in(type): Buffet 
listed_in(city): Bannerghatta Road 
approx_cost(for two people): 750 
cuisines: North Indian, Mughlai, South Indian, Chinese 
dish_liked: Chicken Satay, Mutton Brain, Egg Biryani, Paneer Tikka Masala, Rolls, Lemon Chicken, Butter Chicken
online_order: No 
book_table: No 
votes: 52 
listed_in(type): Cafes 
listed_in(city): Bannerg

In [13]:
df

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city),Description
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\næ A beautiful place to ...",[],Buffet,Banashankari,Experience a delightful culinary journey at ou...
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\næ Had been here for din...",[],Buffet,Banashankari,Discover an exceptional Buffet experience at o...
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\næ Ambience is not that ...",[],Buffet,Banashankari,Embark on a delightful culinary adventure at o...
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\næ Great food and proper...",[],Buffet,Banashankari,Enjoy a delightful Buffet experience at our re...
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\næ Very good restaurant ...",[],Buffet,Banashankari,Experience a delightful Buffet at our restaura...


In [117]:
user_requirements = { 'City': ['Banashankari','Jayanagar'], 'RestaurantType': 'Casual Dining', 'ApproximateCost': 700, 'Cuisine': 'Chinese', 'Votes': 'medium' }

hotels_df = pd.read_csv('data/updated_hotels.csv')

# Extracting user requirements from the input string (assuming it's a dictionary)
# Since the function parameter already seems to be a string, we'll use it directly instead of extracting from a dictionary

filtered_hotels = hotels_df.copy()

#filter hotels based on location
city = user_requirements.get('City')
filtered_hotels = filtered_hotels[filtered_hotels['listed_in(city)'].isin(city)].copy()

# Extracting the cost value from user_requirements and converting it to an integer
if (isinstance(user_requirements.get('ApproximateCost'), str)):
    budget = int(user_requirements.get('ApproximateCost', '0').replace(',', '').split()[0])
    #filtered_hotels['approx_cost(for two people)'] = filtered_hotels['approx_cost(for two people)'].str.replace(',', '').astype(int)
else:
    budget = user_requirements.get('ApproximateCost')

if (budget != 0):
    filtered_hotels = filtered_hotels[filtered_hotels['approx_cost(for two people)'] <= budget].copy()

filtered_hotels['Score'] = 0

for index, row in filtered_hotels.iterrows():
    user_product_match_str = row['hotel_preference']
    hotel_values = user_product_match_str
    hotel_values = { 'City': 'Banashankari', 'RestaurantType': 'Casual Dining', 'ApproximateCost': 700, 'Cuisine': 'Chinese', 'Votes': 'medium' }
    #hotel_values = dictionary_present(user_product_match_str)
    score = 2 #since the location and budget is matched already
    
    # Comparing user requirements with laptop features and updating scores
    for key, user_value in user_requirements.items():
        if key == 'ApproximateCost' or key == 'City':
            continue  # Skipping budget comparison
        hotel_value = hotel_values.get(key, None)
        
        print(hotel_value, user_value)
        if user_value in hotel_value:
            score += 1  # Incrementing score if laptop value meets or exceeds user value
    
    filtered_hotels.loc[index, 'Score'] = score  # Updating the 'Score' column in the DataFrame'''

filtered_hotels['Score']

Casual Dining Casual Dining
Chinese Chinese
medium medium
Casual Dining Casual Dining
Chinese Chinese
medium medium


3    5
4    5
Name: Score, dtype: int64

In [122]:
import pandas as pd
df = pd.read_csv('data/final.csv', encoding='unicode_escape')
df.head(10)

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city),Description
0,https://www.zomato.com/bangalore/rosewood-inte...,"19/1, New Timberyard Layout, Beside Satellite ...",Rosewood International Hotel - Bar & Restaurant,No,No,3.6/5,8,+91 9731716688,Mysore Road,Casual Dining,NaN,"North Indian, South Indian, Andhra, Chinese",800,"[('Rated 5.0', 'RATED\næ Awesome food ??Great ...",[],Buffet,Banashankari,NaN
1,https://www.zomato.com/bangalore/penthouse-caf...,"1, 30th Main Road, 3rd Stage, Banashankari, Ba...",Penthouse Cafe,Yes,No,4.0/5,324,+91 8884135549,Banashankari,Cafe,"Pizza, Mocktails, Coffee, Nachos, Salad, Pasta...","Cafe, Italian, Continental",700,"[('Rated 3.0', ""RATED\næ I had been to this pl...",NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.zomato.com/bangalore/empire-restau...,"4 & 5, Green Orchards Layout, 1st Cross, Banne...",Empire Restaurant,Yes,No,4.3/5,3178,080 49653239,Bannerghatta Road,Casual Dining,"Chicken Satay, Mutton Brain, Egg Biryani, Pane...","North Indian, Mughlai, South Indian, Chinese",750,"[('Rated 5.0', 'RATED\næ Buffet Lunch @ Empire...","['Kerala Parota', 'Chicken Fried Rice', 'Tando...",Buffet,Bannerghatta Road,NaN
4,https://www.zomato.com/bangalore/starbucks-ban...,"Vega City, Dollars Colony, Bannerghatta Road, ...",Starbucks,No,No,3.9/5,52,NaN,Bannerghatta Road,Cafe,NaN,"Cafe, Desserts",700,"[('Rated 5.0', ""RATED\næ The place shuts at 11...",[],Cafes,Bannerghatta Road,NaN
5,https://www.zomato.com/bangalore/timepass-dinn...,"37, 5-1, 4th Floor, Bosco Court, Gandhi Bazaar...",Timepass Dinner,Yes,No,3.8/5,286,+91 9980040002,Basavanagudi,Casual Dining,"Onion Rings, Pasta, Kadhai Paneer, Salads, Sal...",North Indian,600,"[('Rated 3.0', 'RATED\næ Food 3/5\nAmbience 3/...",[],Buffet,Basavanagudi,NaN
6,https://www.zomato.com/bangalore/alchemy-coffe...,"51 Ground Floor, 5th Main Road, 36th Cross Roa...",Alchemy Coffee Roasters,No,No,4.2/5,704,+91 8762473937,Jayanagar,Cafe,"Coffee, Fries, Sandwich, Pizza, Pancakes, Past...",Cafe,500,"[('Rated 4.0', 'RATED\næ Alchemy coffee roaste...",NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,e taste of the food ... nothing special about ...,æ('Rated 4.0',æ'RATED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,æ Wanted to come here from quiet sometime n ya,æhere I was trying some no cheese bread basket,æmushroom soup and South Indian filter coffee....,æ('Rated 3.0',æ'RATED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [126]:
def get_laptop_description(hotel_details):
    delimiter = "#####"

    prompt=f"""
    You are an intelligent marketing  assistant who helps to build better description for the given details on the hotel. You’ll be give some of the properties of the hotel based on which you need to come up with a better description. please note that you'll be given a list of key-value pairs as input
    """
    input = f"""Here is the input json that you need to use to build a good description {hotel_details}"""

    messages=[{"role": "system", "content":prompt },{"role": "user","content":input}]
    #print(messages)

    response = get_chat_completions(messages)
    print('===========',response,'=============')

    return response

In [132]:
def get_chat_completions(input, json_format = False):
    system_message_json_output = """<<. Return output in JSON format to the key output.>>"""

    # If the output is required to be in JSON format
    if json_format == True:
        # Append the input prompt to include JSON response as specified by OpenAI
        input[0]['content'] += system_message_json_output

        # JSON return type specified
        chat_completion_json = openai.ChatCompletion.create(
            model = 'gpt-3.5-turbo-1106',
            messages = input,
            response_format = { "type": "json_object"},
            seed = 1234)

        output = json.loads(chat_completion_json.choices[0].message.content)

    # No JSON return type specified
    else:
        chat_completion = openai.ChatCompletion.create(
            model = 'gpt-3.5-turbo-1106',
            messages = input,
            seed = 2345)

        output = chat_completion.choices[0].message.content

    return output

In [133]:
get_laptop_description('online_order: No book_table: No votes: 8 listed_in(type): Buffet listed_in(city): Banashankari approx_cost(for two people): 800 cuisines: North Indian, South Indian, Andhra, Chinese dish_liked: nan')

=========== Welcome to our hotel located in Banashankari, where you can experience a delightful dining experience! Our restaurant offers a variety of cuisines including North Indian, South Indian, Andhra, and Chinese, ensuring that there is something for everyone to enjoy. With a cozy ambiance and a customer rating of 8 votes, our restaurant provides a comfortable setting for a lovely meal.

Whether you're looking to indulge in a buffet or order a la carte, our menu has something to satisfy every craving. While we do not offer online ordering or table booking, we guarantee a warm welcome and delicious food for an approximate cost of 800 for two people.

Join us for a memorable dining experience and savor the delightful flavors that our diverse menu has to offer! =============


"Welcome to our hotel located in Banashankari, where you can experience a delightful dining experience! Our restaurant offers a variety of cuisines including North Indian, South Indian, Andhra, and Chinese, ensuring that there is something for everyone to enjoy. With a cozy ambiance and a customer rating of 8 votes, our restaurant provides a comfortable setting for a lovely meal.\n\nWhether you're looking to indulge in a buffet or order a la carte, our menu has something to satisfy every craving. While we do not offer online ordering or table booking, we guarantee a warm welcome and delicious food for an approximate cost of 800 for two people.\n\nJoin us for a memorable dining experience and savor the delightful flavors that our diverse menu has to offer!"

In [221]:
def get_laptop_description1(row):
    #print(type(row))
    print(row)
    #print("online_order:", row['online_order'], "\nbook_table:",row['book_table'],"\nvotes:", row['votes'], "\nlisted_in(type):",row['listed_in(type)'], "\nlisted_in(city):",row['listed_in(city)'], "\napprox_cost(for two people):",row['approx_cost(for two people)'], "\ncuisines:",row['cuisines'],"\ndish_liked:",row['dish_liked'])

In [222]:
def updateHotelsCSV():
    hotels_df = pd.read_csv('data/final.csv', encoding='unicode_escape')
    #iterate_llm_response(product_map_layer, hotel_description_1)
    
    ## Create a new column "hotel_preference" that contains the dictionary of the hotel features
    hotels_df['hotel_preference'] = hotels_df.apply(lambda x: get_laptop_description1(x))
    #print(hotels_df)
    #hotels_df.to_csv("updated_hotels.csv",index=False,header = True)

In [223]:
updateHotelsCSV()

0      https://www.zomato.com/bangalore/rosewood-inte...
1      https://www.zomato.com/bangalore/penthouse-caf...
2                                                    NaN
3      https://www.zomato.com/bangalore/empire-restau...
4      https://www.zomato.com/bangalore/starbucks-ban...
                             ...                        
126    https://www.zomato.com/bangalore/lot-like-crep...
127    https://www.zomato.com/bangalore/boondock-bist...
128    https://www.zomato.com/bangalore/blue-tokai-co...
129    https://www.zomato.com/bangalore/oregano-soul-...
130    https://www.zomato.com/bangalore/kalmane-koffe...
Name: url, Length: 131, dtype: object
0      19/1, New Timberyard Layout, Beside Satellite ...
1      1, 30th Main Road, 3rd Stage, Banashankari, Ba...
2                                                    NaN
3      4 & 5, Green Orchards Layout, 1st Cross, Banne...
4      Vega City, Dollars Colony, Bannerghatta Road, ...
                             ...                  

In [225]:
get_laptop_description('online_order: Yes book_table: No votes: 324 listed_in(type): nan listed_in(city): nan approx_cost(for two people): 700 cuisines: Cafe, Italian, Continental dish_liked: Pizza, Mocktails, Coffee, Nachos, Salad, Pasta, Sandwiches')

=========== Introducing a charming café and Italian-Continental bistro, where you can relish delectable dishes and refreshing mocktails. With a diverse menu including pizza, nachos, salads, pasta, and sandwiches, our cozy establishment guarantees a delightful culinary experience. Whether you're in the mood for a casual meal or a flavorful Italian feast, our versatile culinary lineup has something for everyone. Plus, with the convenience of online ordering and an approximate cost for two people at just 700, savoring a memorable dining experience has never been easier. Join us and let your taste buds embark on a delightful journey! =============


"Introducing a charming café and Italian-Continental bistro, where you can relish delectable dishes and refreshing mocktails. With a diverse menu including pizza, nachos, salads, pasta, and sandwiches, our cozy establishment guarantees a delightful culinary experience. Whether you're in the mood for a casual meal or a flavorful Italian feast, our versatile culinary lineup has something for everyone. Plus, with the convenience of online ordering and an approximate cost for two people at just 700, savoring a memorable dining experience has never been easier. Join us and let your taste buds embark on a delightful journey!"

In [228]:
get_laptop_description('online_order: No book_table: No votes: 52 listed_in(type): Cafes listed_in(city): Bannerghatta Road approx_cost(for two people): 700 cuisines: Cafe, Desserts dish_liked: nan')

=========== Experience the cozy charm of our café located in Bannerghatta Road, where you can indulge in delightful café delicacies and tempting desserts. With a casual and inviting ambiance, our café is the perfect spot to relax and unwind with your loved ones. Our menu features a variety of delectable options, and with an approximate cost of 700 for two, it's a delightful escape that won't break the bank. With 52 votes and a focus on quality, our café is a popular choice for locals and visitors alike. While online ordering and table reservations aren't available, we encourage you to drop by and savor the flavors that our café has to offer. =============


"Experience the cozy charm of our café located in Bannerghatta Road, where you can indulge in delightful café delicacies and tempting desserts. With a casual and inviting ambiance, our café is the perfect spot to relax and unwind with your loved ones. Our menu features a variety of delectable options, and with an approximate cost of 700 for two, it's a delightful escape that won't break the bank. With 52 votes and a focus on quality, our café is a popular choice for locals and visitors alike. While online ordering and table reservations aren't available, we encourage you to drop by and savor the flavors that our café has to offer."

In [234]:
import pandas as pd
df = pd.read_csv('data/final.csv', encoding='unicode_escape')
df.head(10)

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city),Description
0,https://www.zomato.com/bangalore/bonum-cibum-h...,"1502, Ground Floor, Sector 1, HSR Layout, HSR,...",Bonum cibum,Yes,No,4.5/5,188,080 22588010,HSR,Casual Dining,"Shawarma, Pasta, Falafel, Mocktails, Babaganus...","Lebanese, Turkish","1,200","[('Rated 1.0', 'RATED\næ This is a restaurant ...","['Roasted Eggplant and Haloumi', 'Falafel', 'H...",Buffet,HSR,NaN
1,https://www.zomato.com/bangalore/lot-like-crep...,"5, Avalon, 1st Main Road, Whitefield Main Road...",Lot Like Crepes & Cafe,Yes,No,4.0 /5,22,+91 9742293475,Whitefield,"Cafe, Dessert Parlor",NaN,"Cafe, Continental, Desserts",600,"[('Rated 5.0', 'RATED\næ Lot like crepes cafe ...",[],Cafes,Whitefield,NaN
2,https://www.zomato.com/bangalore/boondock-bist...,"33, Sundari Armadale, 3rd Floor, Above Axis Ba...",Boondock Bistro,Yes,Yes,3.9 /5,1649,080 49653464,Whitefield,"Casual Dining, Cafe","Burgers, Steak, Pasta, Salad, Chicken Wings, A...","Steak, Continental, American, Italian, Salad, ...",900,"[('Rated 3.0', ""RATED\næ Went to celebrate a f...",[],Cafes,Whitefield,NaN
3,https://www.zomato.com/bangalore/blue-tokai-co...,"8, Upper Ground Floor, VR Bengaluru, Whitefiel...",Blue Tokai Coffee Roasters,Yes,No,4.0 /5,29,+91 7703938506,Whitefield,Cafe,NaN,Cafe,750,"[('Rated 4.0', 'RATED\næ Very good ambiance wi...",[],Cafes,Whitefield,NaN
4,https://www.zomato.com/bangalore/oregano-soul-...,"61, Seetharampalya - Hoodi Road, Whitefield, B...",Oregano Soul Food,Yes,No,4.0 /5,158,080 49653362,Whitefield,Cafe,"Nachos, Sandwich, Coffee, White Sauce Pasta, C...",Cafe,700,"[('Rated 1.0', 'RATED\næ I went with my team i...",[],Cafes,Whitefield,NaN
5,https://www.zomato.com/bangalore/kalmane-koffe...,"28B, Ground Floor, Phoenix Market City Mall, W...",Kalmane Koffees,No,No,3.9 /5,104,080 67266077,Whitefield,Cafe,"Filter Coffee, Blueberry Muffin, Hot Coffee, B...",Cafe,350,"[('Rated 4.0', 'RATED\næ I am a coffee lover. ...",[],Cafes,Whitefield,NaN


In [237]:
for index, row in df.head().iterrows():
    print("===========================")
    x = "online_order:", row['online_order'], " book_table:",row['book_table']," votes:", row['votes'], " listed_in(type):",row['listed_in(type)'], " listed_in(city):",row['listed_in(city)'], " approx_cost(for two people):",row['approx_cost(for two people)'], " cuisines:",row['cuisines']," dish_liked:",row['dish_liked']
    get_laptop_description(x)
    print("===========================")

=========== Welcome to our hotel located in HSR, where you can experience a delectable array of Lebanese and Turkish cuisines. With the convenience of online ordering and a sumptuous buffet listed as one of our features, this is the ideal place to satisfy your cravings. Whether you're a fan of Shawarma, Pasta, Falafel, Mocktails, Babaganush, Pizza, or Chicken Shawarma, our menu is sure to delight. With 188 votes and an approximate cost of $1,200 for two people, our hotel is the perfect spot for a memorable dining experience. =============
=========== Located in the heart of Whitefield, our charming café is the perfect spot for a delightful dining experience. With the option for online ordering, you can easily satisfy your cravings from the comfort of your own home. Indulge in a variety of cuisines including Cafe, Continental, and Desserts, and with an approximate cost of 600 for two people, it's both affordable and delicious. With 22 votes and a variety of dishes to choose from, this c

In [238]:
for index, row in df.tail().iterrows():
    print("===========================")
    x = "online_order:", row['online_order'], " book_table:",row['book_table']," votes:", row['votes'], " listed_in(type):",row['listed_in(type)'], " listed_in(city):",row['listed_in(city)'], " approx_cost(for two people):",row['approx_cost(for two people)'], " cuisines:",row['cuisines']," dish_liked:",row['dish_liked']
    #get_laptop_description(x)
    print("===========================")

=========== Located in the bustling neighborhood of Whitefield, our café is a delightful destination for food enthusiasts. With the convenience of online ordering and a diverse range of cuisines including Cafe, Continental, and Desserts, you're in for a treat. Whether you're looking for a quick bite or a casual dining experience, our approximate cost for two people is budget-friendly at just $600. With a favorable response from 22 votes, our cafe holds its place as a popular choice in the area. While booking a table might not be an option, the delectable dishes will surely make your visit worthwhile. =============
=========== Located in the vibrant city of Whitefield, our cafe is a perfect destination for food enthusiasts. With the convenience of online ordering and table booking, we ensure a seamless dining experience. Renowned for our delectable steak, Continental, American, and Italian cuisines, our menu tantalizes the taste buds of every guest. Whether it's savoring our famous burg

In [243]:
import pandas as pd
df = pd.read_csv('data/updated_hotels.csv', encoding='unicode_escape')
df.head(20)['hotel_preference']

0     {'City': 'Banashankari', 'RestaurantType': ['C...
1     {'City': 'Banashankari', 'RestaurantType': 'Bu...
2     {'City': 'Banashankari', 'RestaurantType': 'Bu...
3     {'City': 'Banashankari', 'RestaurantType': 'Bu...
4     {'City': 'Banashankari', 'RestaurantType': ['B...
5     {'City': 'Banashankari', 'RestaurantType': 'Bu...
6     {'City': 'Bannerghatta Road', 'RestaurantType'...
7     {'City': 'Bannerghatta Road', 'RestaurantType'...
8     {'City': 'Bannerghatta Road', 'RestaurantType'...
9     {'City': 'HSR', 'RestaurantType': 'Buffet', 'A...
10    {'City': 'Whitefield', 'RestaurantType': ['Cas...
11    {'City': 'Whitefield', 'RestaurantType': 'Casu...
12    {'City': 'Whitefield', 'RestaurantType': 'Casu...
13    {'City': 'Whitefield', 'RestaurantType': 'Quic...
14    {'City': 'Whitefield', 'RestaurantType': ['Cas...
Name: hotel_preference, dtype: object